In [107]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

from tqdm import tqdm
import time
from src.scraper import main

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [111]:
input_keywords = urllib.parse.quote(input("Enter skill/s: "))
input_location = urllib.parse.quote(input("Enter location: "))

base_url = f"https://ph.indeed.com/jobs?q={input_keywords}&l={input_location}"
delay = 2


ids = []
titles = []
companies = []
locations = []
descriptions = []
urls=[]

for idx in range(0, 1000, 10):
    # print(idx)
    curr_url = f"{base_url}&start={idx}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
    }

    response = requests.get(url=base_url, headers=headers)
    # print(f"curr_url: {curr_url} returned {response.status_code}")    # debugger

    if response.status_code == 200:
        time.sleep(delay)   # 2 seconds delay
        
        soup = BeautifulSoup(response.content, "html.parser")
        a_elements = soup.find_all('a')
        _ids = [element.get("id") for element in a_elements if element.get("id") and element.get("id").startswith("sj_")]

        for _id in tqdm(_ids):
            url = f"{base_url}&vjk={_id.split('_')[1]}"
            response = requests.get(url=url, headers=headers)

            if response.status_code == 200:
                soup_inner = BeautifulSoup(response.content, "html.parser")

                job_titles = soup_inner.find_all("h2", attrs={"data-testid": "jobsearch-JobInfoHeader-title"})
                job_company = soup_inner.find_all("a", class_="css-1ioi40n e19afand0")
                job_locations = soup_inner.find_all("div", id="jobLocationText")
                job_description = soup_inner.find_all("div", id="jobDescriptionText")

                for title, company, location, description in zip(job_titles, job_company, job_locations, job_description):
                    ids.append(_id)
                    titles.append(title.text.strip())
                    companies.append(company.text.strip())
                    locations.append(location.text.strip())
                    
                    child_elements = description.find_all(recursive=False)
    
                    # Extract text from each child element and join them together
                    description_text = ' '.join(element.get_text(separator=' ', strip=True) for element in child_elements)
                    descriptions.append(description_text)
                    
                    urls.append(url)

100%|██████████| 7/7 [00:04<00:00,  1.41it/s]


In [113]:
jobs_listing = pd.DataFrame({
    "id": ids,
    "title": titles,
    "company": companies,
    "location": locations,
    "description": descriptions,
    "url": urls
})

jobs_listing = jobs_listing.drop_duplicates(subset="id")
jobs_listing = jobs_listing.set_index("id")
jobs_listing

,title,company,location,description,url
id,,,,,
sj_6092e087391b9f6c,Data Analyst Team Leader - job post,Playmate Leisure Solutions Corp,Taguig,About Us: Playmate Leisure Solutions Corp. is ...,https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_c87f6edd0b229722,Reports Analyst - job post,Enshored,Pasig,ESSENTIAL DUTIES AND RESPONSIBILITIES You will...,https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_de6660f6e8faee67,Data Analyst - job post,Visaya Knowledge Process Outsourcing Corporation,Pasig,Responsibilities: Creation of dashboards Gener...,https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_253f29858cd43c5d,Power BI Specialist - job post,"JKC Outsourcing Solutions, Inc.",Manila,"Position Summary: Responsible for designing, d...",https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_f9947544b305fff6,Data Analyst(s) / Engineer(s) - job post,Credit Corp Group Limited,Makati,We are an Australian owned top 200 ASX listed ...,https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_22b3e568add35c48,Payment Lifecycle Analyst - Investigations - j...,JPMorgan Chase & Co,Manila,JOB DESCRIPTION We are looking for a passionat...,https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_67fe7b593107f8fe,Reporting Senior Analyst - job post,Accenture Inc.,Manila,APPLICATION REMINDER: We appreciate your inter...,https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_b8e0991393a4c9dc,Research Analyst - job post,Hakuhodo/BCI,Pasig,• Help develop strategic omnichannel plans tha...,https://ph.indeed.com/jobs?q=data%20analyst&l=...
sj_a3190dadf299d2e2,Market Research Analyst - job post,Casa Bella Home and Living Corp,Pasig,Market Researcher Analyst - Conducts market re...,https://ph.indeed.com/jobs?q=data%20analyst&l=...


In [3]:
keywords = [
    "Sales Associate",
    "Cashier",
    "Merchandisers",
    "Store Assistance",
    "Call Center Agents",
    "Customer Support Relative",
    "Food Server",
    "Kitchen Staff",
    "Hotel Attendants",
    "General Laborers",
    "Helpers",
    "Skilled Trade Assistants",
    "Household Helper",
    "Caregiver",
    "Street Vending",
    "Peddling Goods",
    "Product Line Workers",
    "Machine Operators",
    "Security Guard Services",
    "Security Personnel",
    "Delivery Driver",
    "Warehouse Worker",
    "Packaging",
    "Sorting",
    "Inventory Management",
    "Childcare Assistant",
    "Data Entry Clerk",
    "Entry-Level Tradesperson",
    "Fitness Instructor",
    "Event Staff",
    "Event Setup",
	"Crowd Management",
	"Audio and Visual Support",
    "Photography Assistant",
    "Photographer",
	"Photo Editing",
	"Equipment Setup",
    "Research Assistant",
    "Social Service Aid",
    "Outreach",
	"Resource Coordinator",
	"Client Assistant",
    "Pet Caretaker",
    "Event Planning Assistant",
    "Venue Scouting",
	"Logistics Coordinator",
	"Vendor Management",
	"Onsite Event Support",
]

In [3]:
len(keywords)*50

2400

In [10]:
import tqdm

ModuleNotFoundError: No module named 'tqdm'

In [19]:
import requests
import json
from pathlib import Path
from tqdm import tqdm

BASE_URL = Path("data", "raw")
BASE_URL.mkdir(parents=True, exist_ok=True)

url = "https://linkedin-data-scraper.p.rapidapi.com/search_jobs"

for keyword in tqdm(keywords, desc="Retrieving jobs listings from LinkedIn...", total=len(keywords)):
	payload = {
		"keywords": "Cashier",
		"location": "Philippines",
		"count": 50
	}
	headers = {
		"content-type": "application/json",
		"X-RapidAPI-Key": "16bcea0798msh135696e0d55e6edp1bdd84jsnf81ff6f69482",
		"X-RapidAPI-Host": "linkedin-data-scraper.p.rapidapi.com"
	}

	response = requests.post(url, json=payload, headers=headers)


	if response.status_code == 200:
		_data = response.json()
		_data = _data['response']
		filename = Path(BASE_URL, f"{keyword}.json")

		with open(filename, 'w') as f:
			json.dump(_data, f)

Retrieving jobs listings from LinkedIn...: 100%|██████████| 48/48 [00:08<00:00,  5.88it/s]


In [20]:
files = BASE_URL.glob("*.json")

dfs = list()

for file in files:
    temp_df = pd.read_json(file)
    dfs.append(temp_df)

dfs = pd.concat(dfs, axis=0).reset_index()
dfs.dropna()

,index,title,companyId,companyUniversalName,companyName,salaryInsights,applicants,formattedLocation,formattedEmploymentStatus,formattedExperienceLevel,formattedIndustries,jobDescription,inferredBenefits,jobFunctions,companyApplyUrl,jobPostingUrl,listedAt,comapnyURL1,comapnyURL2
38,38,Cashier,77769340,saversappliances,Savers Appliances,,4.0,"General Mariano Alvarez, Calabarzon, Philippines",Full-time,,,Company Description Savers Appliances is a lea...,,,,https://www.linkedin.com/jobs/view/3903675536/...,2024-04-22T09:40:18.000Z,https://www.linkedin.com/company/77769340,https://www.linkedin.com/company/saversappliances
39,39,Outlet Cashier,3055,accor,Accor,,12.0,"Manila, National Capital Region, Philippines",Full-time,Entry level,Hospitality,Company Description\n\nWhy work for Accor?\n\n...,,OTHR,https://jobs.smartrecruiters.com/AccorHotel/74...,https://www.linkedin.com/jobs/view/3907706552/...,2024-04-22T13:05:49.000Z,https://www.linkedin.com/company/3055,https://www.linkedin.com/company/accor
40,0,"Cashier, Cage",491608,melco-resorts-entertainment,Melco Resorts & Entertainment,,420,Metro Manila,Full-time,,"Entertainment Providers, Gambling Facilities a...","REQ10292 Cashier, Cage (Open Date: 07/02/2024)...",,OTHR,https://melcoresorts.wd3.myworkdayjobs.com/en-...,https://www.linkedin.com/jobs/view/3824198336/...,2024-03-21T09:51:53.000Z,https://www.linkedin.com/company/491608,https://www.linkedin.com/company/melco-resorts...
44,4,Cage Cashier,2907068,solaire-resort,Solaire Resort,,145,"Quezon City, National Capital Region, Philippines",Full-time,Entry level,Travel Arrangements,"Consistently delivers prompt, accurate, courte...",,FIN,https://solaireresort.zohorecruit.com/jobs/Car...,https://www.linkedin.com/jobs/view/3895879629/...,2024-04-12T07:19:27.000Z,https://www.linkedin.com/company/2907068,https://www.linkedin.com/company/solaire-resort
46,6,Outlet Cashier,3055,accor,Accor,,12,"Manila, National Capital Region, Philippines",Full-time,Entry level,Hospitality,Company Description\n\nWhy work for Accor?\n\n...,,OTHR,https://jobs.smartrecruiters.com/AccorHotel/74...,https://www.linkedin.com/jobs/view/3907706552/...,2024-04-22T13:05:49.000Z,https://www.linkedin.com/company/3055,https://www.linkedin.com/company/accor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,27,Branch Regional Cashier,8473766,filinvestgroup,Filinvest Development Corporation,,9,"Bacolod, Western Visayas, Philippines",Full-time,Entry level,Investment Management,Will be assigned to accept trade and non-trade...,,OTHR,https://www.careers-page.com/filinvest-develop...,https://www.linkedin.com/jobs/view/3886648526/...,2024-04-08T16:29:00.000Z,https://www.linkedin.com/company/8473766,https://www.linkedin.com/company/filinvestgroup
948,28,Administrative Assistant & Cashier - Hampton O...,8473766,filinvestgroup,Filinvest Development Corporation,,13,"San Fernando, Central Luzon, Philippines",Full-time,Entry level,Investment Management,ADMINISTRATIVE AND CASHIER ASSISTANT\n\nThe Ad...,,ADM,https://www.careers-page.com/filinvest-develop...,https://www.linkedin.com/jobs/view/3845614937/...,2024-03-04T04:29:02.000Z,https://www.linkedin.com/company/8473766,https://www.linkedin.com/company/filinvestgroup
950,30,Cashier,91308533,paramount-human-resource-multi-purpose-coopera...,Paramount Human Resource Multi-Purpose Coopera...,,17,"Pasay, National Capital Region, Philippines",Full-time,Entry level,Human Resources Services,Qualifications\n\nWith 6 months to 1-year rela...,,OTHR,https://ph.jobslin.com/job/108384/cashier-in-n...,https://www.linkedin.com/jobs/view/3850548561/...,2024-03-08T00:00:00.000Z,https://www.linkedin.com/company/91308533,https://www.linkedin.com/company/paramount-hum...
955,35,Cashier - Trabajo Espana,18173818,tamsi,Top Asia,,17,"Quezon City, National Capital Region, Philippines",Full-time,Entry level,Human Resources Services,About Us\n\nTop Asia is a leading Food and Bev...,,ACCT,https://careers.topasiamanagement.com/

In [ ]:
import requests



url = "https://jsearch.p.rapidapi.com/search"

querystring = {"query":"Call Center Agent in Philippines","page":"1","num_pages":"1"}

headers = {
	"X-RapidAPI-Key": "aacc989fcbmsh273b7841567d07cp15970djsn04a3ea8bdf91",
	"X-RapidAPI-Host": "jsearch.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

In [35]:
url = "https://jsearch.p.rapidapi.com/search"

BASE_URL = Path("data", "raw2")
BASE_URL.mkdir(parents=True, exist_ok=True)


for keyword in tqdm(keywords, desc="Retrieving jobs listings...", total=len(keywords)):
	
    querystring = {"query":f"{keyword} in Philippines","page":"1","num_pages":"20"}

    headers = {
        "X-RapidAPI-Key": "aacc989fcbmsh273b7841567d07cp15970djsn04a3ea8bdf91",
        "X-RapidAPI-Host": "jsearch.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        _data = response.json()
        _data = _data['data']

        filename = Path(BASE_URL, f"{keyword}.json")

        with open(filename, 'w') as f:
            json.dump(_data, f)

Retrieving jobs listings...: 100%|██████████| 48/48 [02:28<00:00,  3.09s/it]


In [27]:
temp = pd.DataFrame(_data)

In [34]:
print(temp['job_description'][0])

• Bachelor’s degree in business or a related field.
• Excellent selling, negotiation and communication skills.
• Prioritizing, time management and organizational skills
• Excellent knowledge of MS Office.
• Can communicate and understand English and Tagalog language.
• Applicants must be willing to work in Makati City.

Job Type: Full-time

Pay: Php15,000.00 - Php20,000.00 per month

Schedule:
• 8 hour shift

Supplemental pay types:
• 13th month salary
• Commission pay

Education:
• Bachelor's (Preferred)

Experience:
• Sales Associate: 1 year (Required)

Language:
• English (Required)


In [39]:
files = BASE_URL.glob("*.json")

dfs = list()

for file in files:
    temp_df = pd.read_json(file)
    dfs.append(temp_df)

dfs = pd.concat(dfs, axis=0).reset_index()
dfs['index'].columns = 'id'

,index,job_id,employer_name,employer_logo,employer_website,employer_company_type,job_publisher,job_employment_type,job_title,job_apply_link,...,job_salary_currency,job_salary_period,job_highlights,job_job_title,job_posting_language,job_onet_soc,job_onet_job_zone,job_occupational_categories,job_naics_code,job_naics_name
0,0,KL-RCyYTIifr7pWGAAAAAA==,Smart Apartment Data,https://encrypted-tbn0.gstatic.com/images?q=tb...,None,None,Glassdoor,FULLTIME,"Call Center Agent - Cebu, Philippines based | ...",https://www.glassdoor.com/job-listing/call-cen...,...,None,None,{},Center agent,en,43405100.0,2.0,None,NaN,None
1,1,BYiEwJcIk6urFMi3AAAAAA==,BPO GLOBAL,None,None,None,Indeed,FULLTIME,Call Center Agent/Customer Service Representat...,https://ph.indeed.com/viewjob?jk=0440fd056d9b98a2,...,PHP,MONTH,{},Service representative,en,43405100.0,2.0,None,NaN,None
2,2,qK3BWj1N0lPlPQZsAAAAAA==,Six Eleven Global Teleservices,None,None,None,SmartRecruiters Job Search,FULLTIME,Urgent Hiring for Call Center Agents,https://jobs.smartrecruiters.com/SixElevenGlob...,...,None,None,{},None,en,43405100.0,2.0,None,NaN,None
3,3,ut7MsuQj3sN5I2zyAAAAAA==,Qualfon Philippines Incorporated,https://encrypted-tbn0.gstatic.com/images?q=tb...,None,None,Glassdoor,FULLTIME,Call Center Agent,https://www.glassdoor.com/job-listing/call-cen...,...,None,None,{},Center agent,en,43405100.0,2.0,None,NaN,None
4,4,4peGen0lrvkaNCp-AAAAAA==,GH Specialist Philippines Incorporated,None,None,None,Glassdoor,FULLTIME,Call Center Agents - Up to 24K Salary,https://www.glassdoor.com/job-listing/call-cen...,...,None,None,{},Call center,en,43101100.0,3.0,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,96,CdMfrq4CBRdm-DBKAAAAAA==,"Sateco Philippines, Inc.",None,None,None,Glassdoor,FULLTIME,Warehouse Supervisor,https://www.glassdoor.com/job-listing/warehous...,...,None,None,{},Warehouse supervisor,en,53102100.0,2.0,None,NaN,None
2024,97,C2sXcd35AyXtDfTaAAAAAA==,Lazada,https://laz-img-cdn.alicdn.com/images/ims-web/...,http://www.lazada.com,Retail,Indeed,FULLTIME,Choice Warehouse Operation & Solution Supervis...,https://ph.indeed.com/viewjob?jk=2c25c8561de1526d,...,None,None,{},None,en,53102100.0,2.0,None,454111.0,Electronic Shopping
2025,98,ni4erYsRdEBL7fDbAAAAAA==,Career Connect,https://encrypted-tbn0.gstatic.com/images?q=tb...,http://www.careerconnect.org.uk,None,GrabJobs,FULLTIME,Warehouse Supervisor,https://grabjobs.co/philippines/job/full-time/...,...,None,None,{},Warehouse supervisor,en,53102100.0,2.0,[Warehouse Associate],NaN,None
2026,99,3OALAXNVHx1_QiyzAAAAAA==,PC EXPRESS,None,None,None,Jobrapido.com,FULLTIME,Warehouse Driver,https://ph.jobrapido.com/jobpreview/29824899,...,None,None,{},Driver,en,53303300.0,2.0,None,NaN,None


In [47]:
text = dfs["job_description"].iloc[2026]

In [49]:
print(text)

JOIN OUR TEAM!

PC EXPRESS is the leading computer retailer in the Philippines.

We are currently looking for a WAREHOUSE DRIVER.

Qualifications:1. High school graduate, college level or college graduate.2. Trustworthy, flexible, willing to work long hours, in good health and physically able to perform all duties of a driver.3. Must be well-versed in Iloilo routes.4. With 1 – 3 years experience.5. Must possess a professional driver’s license with Restriction Code 123.

Summary of role requirements:
• Flexible hours available
• 2-3 years of relevant work experience required for this role
• Working rights required for this role
• Expected start date for role: 29 April 2024
• Expected salary: ₱12,200 per month


In [51]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove non-alphanumeric characters and extra whitespaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    
    # Join the tokens back into a string
    preprocessed_text = ' '.join(lemmatized_tokens)
    
    return preprocessed_text

# Preprocess the text
preprocessed_text = preprocess_text(text)

print("Original Text:")
print(text)
print("\nPreprocessed Text:")
print(preprocessed_text)

# Vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform([preprocessed_text])

print("\nTF-IDF Matrix:")
print(tfidf_matrix.toarray())


Original Text:
JOIN OUR TEAM!

PC EXPRESS is the leading computer retailer in the Philippines.

We are currently looking for a WAREHOUSE DRIVER.

Qualifications:1. High school graduate, college level or college graduate.2. Trustworthy, flexible, willing to work long hours, in good health and physically able to perform all duties of a driver.3. Must be well-versed in Iloilo routes.4. With 1 – 3 years experience.5. Must possess a professional driver’s license with Restriction Code 123.

Summary of role requirements:
• Flexible hours available
• 2-3 years of relevant work experience required for this role
• Working rights required for this role
• Expected start date for role: 29 April 2024
• Expected salary: ₱12,200 per month

Preprocessed Text:
join team pc express leading computer retailer philippine currently looking warehouse driver qualifications1 high school graduate college level college graduate2 trustworthy flexible willing work long hour good health physically able perform duty 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\drchr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\drchr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\drchr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
len(tfidf_matrix.toarray()[0])

64

In [61]:
len(preprocessed_text)

561

In [67]:
threshold = 10
columns_to_drop = dfs.columns[dfs.isnull().sum() > threshold]

# Dropping columns
final_df = dfs.drop(columns=columns_to_drop)

In [76]:
final_df

,index,job_id,employer_name,job_publisher,job_title,job_apply_link,job_apply_is_direct,job_apply_quality_score,apply_options,job_description,job_is_remote,job_google_link,job_required_experience,job_required_education,job_experience_in_place_of_education,job_highlights,job_posting_language,job_onet_soc
0,0,KL-RCyYTIifr7pWGAAAAAA==,Smart Apartment Data,Glassdoor,"Call Center Agent - Cebu, Philippines based | ...",https://www.glassdoor.com/job-listing/call-cen...,True,0.6559,"[{'publisher': 'Glassdoor', 'apply_link': 'htt...",Outbound Call Center Agent\n\nThis is a work r...,1.0,https://www.google.com/search?gl=us&hl=en&rciv...,"{'no_experience_required': False, 'required_ex...","{'postgraduate_degree': False, 'professional_c...",False,{},en,43405100.0
1,1,BYiEwJcIk6urFMi3AAAAAA==,BPO GLOBAL,Indeed,Call Center Agent/Customer Service Representat...,https://ph.indeed.com/viewjob?jk=0440fd056d9b98a2,True,0.6182,"[{'publisher': 'Indeed', 'apply_link': 'https:...",Qualifications\n- Must have 2 years of accumul...,1.0,https://www.google.com/search?gl=us&hl=en&rciv...,"{'no_experience_required': False, 'required_ex...","{'postgraduate_degree': False, 'professional_c...",False,{},en,43405100.0
2,2,qK3BWj1N0lPlPQZsAAAAAA==,Six Eleven Global Teleservices,SmartRecruiters Job Search,Urgent Hiring for Call Center Agents,https://jobs.smartrecruiters.com/SixElevenGlob...,False,0.7435,"[{'publisher': 'SmartRecruiters Job Search', '...",Company Description\n\nWe are a premier call c...,0.0,https://www.google.com/search?gl=us&hl=en&rciv...,"{'no_experience_required': True, 'required_exp...","{'postgraduate_degree': False, 'professional_c...",False,{},en,43405100.0
3,3,ut7MsuQj3sN5I2zyAAAAAA==,Qualfon Philippines Incorporated,Glassdoor,Call Center Agent,https://www.glassdoor.com/job-listing/call-cen...,True,0.6599,"[{'publisher': 'Glassdoor', 'apply_link': 'htt...","Our company is Qualfon. Currently, we are hiri...",0.0,https://www.google.com/search?gl=us&hl=en&rciv...,"{'no_experience_required': True, 'required_exp...","{'postgraduate_degree': False, 'professional_c...",False,{},en,43405100.0
4,4,4peGen0lrvkaNCp-AAAAAA==,GH Specialist Philippines Incorporated,Glassdoor,Call Center Agents - Up to 24K Salary,https://www.glassdoor.com/job-listing/call-cen...,True,0.6653,"[{'publisher': 'Glassdoor', 'apply_link': 'htt...",QUALIFICATIONS\n• Preferably with call center ...,0.0,https://www.google.com/search?gl=us&hl=en&rciv...,"{'no_experience_required': False, 'required_ex...","{'postgraduate_degree': False, 'professional_c...",False,{},en,43101100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,96,CdMfrq4CBRdm-DBKAAAAAA==,"Sateco Philippines, Inc.",Glassdoor,Warehouse Supervisor,https://www.glassdoor.com/job-listing/warehous...,True,0.6492,"[{'publisher': 'Glassdoor', 'apply_link': 'htt...",· Ensure the proper implementation and observa...,0.0,https://www.google.com/search?gl=us&hl=en&rciv...,"{'no_experience_required': False, 'required_ex...","{'postgraduate_degree': False, 'professional_c...",False,{},en,53102100.0
2024,97,C2sXcd35AyXtDfTaAAAAAA==,Lazada,Indeed,Choice Warehouse Operation & Solution Supervis...,https://ph.indeed.com/viewjob?jk=2c25c8561de1526d,False,0.6274,"[{'publisher': 'Indeed', 'apply_link': 'https:...",Location:\n\nPhilippines\n\nDepartment: Supply...,0.0,https://www.google.com/search?gl=us&hl=en&rciv...,"{'no_experience_required': False, 'required_ex...","{'postgraduate_degree': False, 'professional_c...",False,{},en,53102100.0
2025,98,ni4erYsRdEBL7fDbAAAAAA==,Career Connect,GrabJobs,Warehouse Supervisor,https://grabjobs.co/philippines/job/full-time/...,True,0.5610,"[{'publisher': 'GrabJobs', 'apply_link': 'http...",Position Summary:\n\nThe Warehouse Supervisor ...,0.0,https://www.google.com/search?gl=us&hl=en&rciv...,"{'no_experience_required': False, 'required_ex...","{'postgraduate_degree': False, 'professional_c...",False,{},en,53102100.0
2026,99,3OALAXNVHx1_QiyzAAAAAA==,PC EXPRESS,Jobrapi

In [73]:
final_df.columns

Index(['index', 'job_id', 'employer_name', 'job_publisher', 'job_title',
       'job_apply_link', 'job_apply_is_direct', 'job_apply_quality_score',
       'apply_options', 'job_description', 'job_is_remote', 'job_google_link',
       'job_required_experience', 'job_required_education',
       'job_experience_in_place_of_education', 'job_highlights',
       'job_posting_language', 'job_onet_soc'],
      dtype='object')

In [78]:
dest_dir = Path("data", "processed", "text")
dest_dir.mkdir(parents=True, exist_ok=True)
final_df.to_csv(Path(dest_dir, "job-listings.csv"))